In [1]:
import os
import sys
import multiprocessing

sys.path.append(os.path.join('..', '..'))

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (10.0, 10.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy as np
import pandas as pd
import logging
import time
import pickle

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import constants as const
from astropy.wcs.utils import pixel_to_skycoord

from arl.data.polarisation import PolarisationFrame
from arl.visibility.base import create_visibility
from arl.skycomponent.operations import create_skycomponent
from arl.image.operations import show_image, import_image_from_fits, export_image_to_fits, \
    smooth_image, calculate_image_frequency_moments, calculate_image_from_frequency_moments
from arl.image.deconvolution import deconvolve_cube, restore_cube
from arl.image.iterators import  image_raster_iter
from arl.image.solvers import solve_image
from arl.visibility.iterators import vis_timeslice_iter
# from arl.util.testing_support import create_named_configuration, \
#     create_low_test_image_from_gleam, create_low_test_beam
from arl.imaging import *
from arl.imaging.weighting import weight_visibility

from IPython.display import display, HTML
import plotly.offline as py

from help.array import add_arr_sum
from help.plot import plot_table_bar, show

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

py.init_notebook_mode(connected=True)

In [2]:
# construct low configuration
results_dir = './results'
data_dir = os.path.join('./examples/arl', results_dir)  # for import/export
os.makedirs(results_dir, exist_ok=True)

# low = create_named_configuration('LOWBD2-CORE')
cellsize = 0.001
npixel=512
padding = 2
oversampling = 2
nchan = 2
times = np.linspace(-3, +3, 3) * np.pi / 12.0
frequency = np.linspace(0.8e8, 1.2e8, nchan)
ntimes, nfreq = len(times), len(frequency)
centre_frequency = np.array([np.average(frequency)])
channel_bandwidth=np.array(nchan * [frequency[1]-frequency[0]])
total_bandwidth = np.array([np.sum(channel_bandwidth)])
polarisation_frame = PolarisationFrame('stokesI')

In [3]:
# create visibility
vt = pickle.load(open('%s/visibility2.vis' % results_dir, 'rb'))

In [4]:
# make a test image
model = import_image_from_fits('%s/model_centrechannel2.fits' % data_dir)

import_image_from_fits: created >f8 image of shape (1, 1, 512, 512), size 0.002 (GB)
import_image_from_fits: Max, min in ./examples/arl/./results/model_centrechannel2.fits = 40.971054, -0.127373


In [5]:
# predict
def test_predict(predict, vt, model):
    vt.data['vis'] *= 0.0

    time_pred0 = -time.time()
    vt0 = predict(vt, model, nprocessor=1, timeslice='auto', vis_slices=1,
        oversampling=oversampling, facets=2, wstep=50, polarisation_frame=polarisation_frame)
    time_pred0 += time.time()

    time_pred1 = -time.time()
    vt1 = predict(vt, model, nprocessor=1, timeslice='auto', vis_slices=1,
        oversampling=oversampling, facets=2, wstep=50, polarisation_frame=polarisation_frame, opt=True)
    time_pred1 += time.time()

    vt = vt0
    
    display(HTML('<h4>Original  predict time: {:.2f}s'.format(time_pred0)))
    display(HTML('<h4>Optimized predict time: {:.2f}s'.format(time_pred1)))
    
    return vt, time_pred0, time_pred1

In [6]:
# invert
def test_invert(invert, vt, model):
    time_inv0 = -time.time()
    dirty0, sumwt0 = invert(vt, model, padding=1, nprocessor=1, timeslice='auto', vis_slices=1,
        oversampling=oversampling, facets=2, wstep=50, polarisation_frame=polarisation_frame)
    psf0, sumwt0 = invert(vt, model, dopsf=True, padding=1, nprocessor=1, 
        timeslice='auto', vis_slices=1, oversampling=oversampling, facets=2, wstep=50, 
        polarisation_frame=polarisation_frame)
    time_inv0 += time.time()

    time_inv1 = -time.time()
    dirty1, sumwt1 = invert(vt, model, padding=1, nprocessor=1, timeslice='auto', vis_slices=1,
        oversampling=oversampling, facets=2, wstep=50, polarisation_frame=polarisation_frame, opt=True)
    psf1, sumwt1 = invert(vt, model, dopsf=True, padding=1, nprocessor=1, 
        timeslice='auto', vis_slices=1, oversampling=oversampling, facets=2, wstep=50, 
        polarisation_frame=polarisation_frame, opt=True)
    time_inv1 += time.time()

    dirty, psf, sumwt = dirty0, psf0, sumwt0
    
    display(HTML('<h4>Original  invert time: {:.2f}s'.format(time_inv0)))
    display(HTML('<h4>Optimized invert time: {:.2f}s'.format(time_inv1)))
    
    return dirty, psf, sumwt, time_inv0, time_inv1

In [7]:
# predict and invert algorithm
predict_processors = [predict_2d, predict_wstack, predict_timeslice, predict_facets, predict_wprojection]
invert_processors =  [invert_2d,  invert_wstack,  invert_timeslice,  invert_facets,  invert_wprojection]

time_pred_ori, time_pred_opt = [], []
time_inv_ori,  time_inv_opt  = [], []

for predict, invert in zip(predict_processors, invert_processors):
    vt, time_pred0, time_pred1 = test_predict(predict, vt, model)
    dirty, psf, sumwt, time_inv0, time_inv1 = test_invert(invert, vt, model)
    
    time_pred_ori.append(time_pred0)
    time_pred_opt.append(time_pred1)
    time_inv_ori.append(time_inv0)
    time_inv_opt.append(time_inv1)

predict_2d: predict using 2d transform
Using original algorithm
shift_vis_from_image: shifting phasecentre from image phase centre <SkyCoord (ICRS): (ra, dec) in deg
    ( 15., -35.)> to visibility phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 15., -45.)>
predict_2d: predict using 2d transform
Using optimized algorithm
shift_vis_from_image: shifting phasecentre from image phase centre <SkyCoord (ICRS): (ra, dec) in deg
    ( 15., -35.)> to visibility phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 15., -45.)>


invert_2d: inverting using 2d transform
Using original algorithm
invert_2d: inverting using 2d transform
Using original algorithm
invert_2d: inverting using 2d transform
Using optimized algorithm
invert_2d: inverting using 2d transform
Using optimized algorithm


predict_wstack: predicting using wstack
predict_with_vis_iterator: Processing chunks
predict_wstack_single: predicting using single w slice
create_w_term_image: For w = -39.3, field of view = 0.512000, Fresnel number = 2.58
Using original algorithm
Using original algorithm
predict_wstack: predicting using wstack
predict_with_vis_iterator: Processing chunks
predict_wstack_single: predicting using single w slice
create_w_term_image: For w = -39.3, field of view = 0.512000, Fresnel number = 2.58
Using optimized algorithm
Using optimized algorithm


invert_wstack: inverting using wstack
invert_wstack_single: predicting using single w slice
Using original algorithm
invert_2d_base: retaining imaginary part of dirty image
create_w_term_image: For w = -39.3, field of view = 0.512000, Fresnel number = 2.58
invert_wstack: inverting using wstack
invert_wstack_single: predicting using single w slice
Using original algorithm
invert_2d_base: retaining imaginary part of dirty image
create_w_term_image: For w = -39.3, field of view = 0.512000, Fresnel number = 2.58
invert_wstack: inverting using wstack
invert_wstack_single: predicting using single w slice
Using optimized algorithm
invert_2d_base: retaining imaginary part of dirty image
create_w_term_image: For w = -39.3, field of view = 0.512000, Fresnel number = 2.58
invert_wstack: inverting using wstack
invert_wstack_single: predicting using single w slice
Using optimized algorithm
invert_2d_base: retaining imaginary part of dirty image
create_w_term_image: For w = -39.3, field of view = 0.

predict_timeslice: predicting using time slices
predict_with_vis_iterator: Processing chunks
predict_timeslice_single: predicting using single time slice
fit_uvwplane: Fit to 54780 rows reduces rms w from 47.5 to 35.5 m
Using original algorithm
predict_timeslice: predicting using time slices
predict_with_vis_iterator: Processing chunks
predict_timeslice_single: predicting using single time slice
fit_uvwplane: Fit to 54780 rows reduces rms w from 47.5 to 35.5 m
Using optimized algorithm


...

predict_facets: Predicting by image facets
predict_with_image_iterator: Predicting by image partitions
raster: predicting using 2 x 2 image partitions
raster: spacing of raster (256, 256)
raster: partition (0, 0) of (2, 2)
Using original algorithm
shift_vis_from_image: shifting phasecentre from image phase centre <SkyCoord (ICRS): (ra, dec) in deg
    ( 24.91597536, -41.98624265)> to visibility phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 15., -35.)>
raster: partition (1, 0) of (2, 2)
Using original algorithm
shift_vis_from_image: shifting phasecentre from image phase centre <SkyCoord (ICRS): (ra, dec) in deg
    ( 5.08402464, -41.98624265)> to visibility phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 15., -35.)>
raster: partition (0, 1) of (2, 2)
Using original algorithm
shift_vis_from_image: shifting phasecentre from image phase centre <SkyCoord (ICRS): (ra, dec) in deg
    ( 23.2848329, -27.33867206)> to visibility phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 1

invert_facets: Inverting by image facets
invert_with_image_iterator: Inverting by image partitions
raster: predicting using 2 x 2 image partitions
raster: spacing of raster (256, 256)
raster: partition (0, 0) of (2, 2)
Using original algorithm
shift_vis_from_image: shifting phasecentre from vis phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 15., -35.)> to image phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 24.91597536, -41.98624265)>
raster: partition (1, 0) of (2, 2)
Using original algorithm
shift_vis_from_image: shifting phasecentre from vis phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 15., -35.)> to image phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 5.08402464, -41.98624265)>
raster: partition (0, 1) of (2, 2)
Using original algorithm
shift_vis_from_image: shifting phasecentre from vis phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 15., -35.)> to image phasecentre <SkyCoord (ICRS): (ra, dec) in deg
    ( 23.2848329, -27.33867206)>
raster: partition

predict_wprojection: predict using wprojection
Using original algorithm
get_kernel_list: Using wprojection kernel
get_kernel_list: Fresnel number = 65.536000
advise_wide_field: Maximum wavelength 3.747 (meters)
advise_wide_field: Minimum wavelength 2.498 (meters)
advise_wide_field: Maximum baseline 315.2 (wavelengths)
advise_wide_field: Station/antenna diameter 35.0 (meters)
advise_wide_field: Primary beam 0.107069 (rad) 6.135 (deg)
advise_wide_field: Image field of view 0.642412 (rad) 36.808 (deg)
advise_wide_field: Synthesized beam 0.003173 (rad) 0.182 (deg)
advise_wide_field: Cellsize 0.001058 (rad) 0.061 (deg)
advice_wide_field: Npixels per side = 607
advice_wide_field: Npixels (power of 2, 3) per side = 768
advice_wide_field: W sampling for full image = 0.2 (wavelengths)
advice_wide_field: W sampling for primary beam = 5.6 (wavelengths)
advice_wide_field: Time sampling for full image = 13.5 (s)
advice_wide_field: Time sampling for primary beam = 484.6 (s)
advice_wide_field: Freque

invert_2d: inverting using wprojection
Using original algorithm
get_kernel_list: Using wprojection kernel
get_kernel_list: Fresnel number = 65.536000
advise_wide_field: Maximum wavelength 3.747 (meters)
advise_wide_field: Minimum wavelength 2.498 (meters)
advise_wide_field: Maximum baseline 315.2 (wavelengths)
advise_wide_field: Station/antenna diameter 35.0 (meters)
advise_wide_field: Primary beam 0.107069 (rad) 6.135 (deg)
advise_wide_field: Image field of view 0.642412 (rad) 36.808 (deg)
advise_wide_field: Synthesized beam 0.003173 (rad) 0.182 (deg)
advise_wide_field: Cellsize 0.001058 (rad) 0.061 (deg)
advice_wide_field: Npixels per side = 607
advice_wide_field: Npixels (power of 2, 3) per side = 768
advice_wide_field: W sampling for full image = 0.2 (wavelengths)
advice_wide_field: W sampling for primary beam = 5.6 (wavelengths)
advice_wide_field: Time sampling for full image = 13.5 (s)
advice_wide_field: Time sampling for primary beam = 484.6 (s)
advice_wide_field: Frequency samp

get_kernel_list: Maximum w kernel full width = 52 pixels
w_kernel_list: Maximum absolute w = 203.6, step is 50.0 wavelengths
==== 10 50 10
create_w_term_image: For w = -203.6, field of view = 0.512000, Fresnel number = 13.34
create_w_term_image: For w = -158.3, field of view = 0.512000, Fresnel number = 10.38
create_w_term_image: For w = -113.1, field of view = 0.512000, Fresnel number = 7.41
create_w_term_image: For w = -67.9, field of view = 0.512000, Fresnel number = 4.45
create_w_term_image: For w = -22.6, field of view = 0.512000, Fresnel number = 1.48
create_w_term_image: For w = 22.6, field of view = 0.512000, Fresnel number = 1.48
create_w_term_image: For w = 67.9, field of view = 0.512000, Fresnel number = 4.45
create_w_term_image: For w = 113.1, field of view = 0.512000, Fresnel number = 7.41
create_w_term_image: For w = 158.3, field of view = 0.512000, Fresnel number = 10.38
create_w_term_image: For w = 203.6, field of view = 0.512000, Fresnel number = 13.34


In [8]:
# Imaging algorithm

# Predict

origin = np.around(time_pred_ori, decimals=2)
optimized = np.around(time_pred_opt, decimals=2)
columns = ['2D', 'Wstack', 'Timeslice', 'Facets', 'WProject']
fig_title = 'Imaging algorithm (Predict)'
show(origin, optimized, columns, fig_title)

# Invert
origin = np.around(time_inv_ori, decimals=2)
optimized = np.around(time_inv_opt, decimals=2)
columns = ['2D', 'Wstack', 'Timeslice', 'Facets', 'WProject']
fig_title = 'Imaging algorithm (Invert)'
show(origin, optimized, columns, fig_title)